In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'multdata:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5172126%2F8637046%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240730%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240730T121017Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7792ca2089af8e75af5a51791b223dc369f9ff3c58d738516845a671a33a6553c304614c571f40af346d37af20d8d2d86b3a1eb272aa146f0481d80e45efb21481d0ff145b0e10c3c5032133f474ab989f1e76d3aee28fda21a48513e8700060dc24b6e55d97daf837a00a5b9e99cc24c7d3a6b2402c3a8351e6025bb2b03fa0600d41183c112fc944590a9dac55651e2f351571f03eca3b5ed93d121b1407c329ef93bacb3101aa0bcd6cc60f58cc75ff80610a02bc6f9b4a8cb4f031a5bff5baaeaeff4a627a0ec6d0686b3321b0364d48d88407d19a75a43691bdaab0e883353a74bc9f64e9979a03fe461ae6ef588604df7cd17458787a7a0ae0af2b5c0d,500-mult:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5209521%2F8688482%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240730%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240730T121017Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D79ac9bcf8f90117d89392be14ee978e3cb8d41b09c9e452e0bc73cf5ea9d61f7c5f502e6c38dcd0186f954f0468fca2c54dfdc5bd6b000d8cb7553d75e07551f8636879e5d63ffcff5ae47188ea484464bb5b2acc773b87d392ad4c68398b5e6ec87066d175c091c0575741f9772dcfa914238cb32fae77a7b601d8ec1bc76e14a656851aac3e36ddc0b0b52848c8d3d58410b1e7f03db7b95e648bf0f5adfbe711e3ba0abcae2d9d05f735b89760ba230a45ad69e92477683c6e0868901b1810b86f131765cd3de1a8fbc60ab4dad613f6b38fe2297f5814072e360587fa3ff6a8b8c553f154ef8c4656f386bfccebed73aee85ee4623e10d966f243c1697eb,topics22k:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5358110%2F8911094%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240730%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240730T121017Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D17be7d1225cd2b19cf54eb2bec52f4b7bbfd745bff96b4f0a2f86a56b2dca11c456427868299dee491b256bec7e8e756844d0d837286cd60bb95fafa3baeb8aec61f9854dc1c3edf618732374790ef37d0b9dabbf0647b746c3fa0c81dfa3d3359750c968d8f30ddc8a745527fb2204e6f55543f2e2598dd796ec7619aacd3267aa65fdd2bf29a8fe196244278a481af305b55f7c9a0e0f2900bcd36191fe765a7bcedb9bed642b9dcf935ff82a023d6f9c44bf0b1ef8e12f28f976c8bb8a6b7c0d236a89648988a19d4a439c7b6a0a58a6a1cbf23ee55c990c832b40f92f21e82a706d7764723b5c84354045406434d0683e0331a286c73100986f65db89905,new-topics:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5358953%2F8912142%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240730%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240730T121017Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6952d6b64b7fa05cdf93d7e362cc9b61acfdf9d9843f8413be5ad37dd3bcf1a0843a291e9293bf0f372af7be9ced969c848b3272f5da0fcbb208afadd3d9068f2d0bf347500576d52c62a2d6c810684917761e703bd8664c50a5fee10bb11fb92a289571f4dfe44d4aa4115cd58bf40cc6db7f8470cbcd49de0ca2ad01468e5204430cd5e654444786116f07c175b53a4911f4f87c81b54ffc2ea627994b711879b600f174097341a8b8ed604e2f8cd8758a407cd73b4d383e6698ab5378a98298e1e06fb7bd6eb7873f148d524cabdeda28965d3617f314c657fa81ba9995db0f844b22cd51313a3d1d6cd36c0b0da21b37f420c2b90fdff86a90fdcd93e67b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/500-mult/500.txt
/kaggle/input/multdata/_mult.txt
/kaggle/input/new-topics/_new_topics.txt
/kaggle/input/topics22k/_topics.txt


In [ ]:
!pip install langdetect
!pip install tiktoken
import tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=4e0e0fe79c01a7fd782003065ce039a446ec4a4467a5910031449f7885b99065
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00a 0:00:01


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Initialize the tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-4o')

# Load dataset
file_path = "/kaggle/input/multdata/_mult.txt"
# file_path = "/kaggle/input/new-topics/_new_topics.txt"
dataset = load_dataset('text', data_files=file_path)
train_dataset, val_dataset = dataset['train'].train_test_split(test_size=0.1)['train'], dataset['train'].train_test_split(test_size=0.1)['test']

# Function to tokenize examples and pad sequences manually
def tokenize_and_pad(examples, max_length):
    tokenized_examples = []
    for example in examples:
        tokens = tokenizer.encode(example)
        attention_mask = [1] * len(tokens)
        padded_tokens = tokens + [0] * (max_length - len(tokens))  # Pad with zeros
        padded_attention_mask = attention_mask + [0] * (max_length - len(attention_mask))
        tokenized_examples.append({
            "input_ids": padded_tokens,
            "attention_mask": padded_attention_mask,
            "labels": padded_tokens.copy()  # Use input_ids as labels for language modeling
        })

    return tokenized_examples

# Determine maximum sequence length
train_max_length = max(len(tokenizer.encode(example)) for example in train_dataset['text'])
val_max_length = max(len(tokenizer.encode(example)) for example in val_dataset['text'])
max_length = max(train_max_length, val_max_length)

print("Max length:", max_length)

# Tokenize and pad train and validation datasets manually
tokenized_train_examples = tokenize_and_pad(train_dataset['text'], max_length)
tokenized_val_examples = tokenize_and_pad(val_dataset['text'], max_length)


# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenized_examples):
        self.examples = tokenized_examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.examples[idx].items()}
        return item

train_dataset = CustomDataset(tokenized_train_examples)
val_dataset = CustomDataset(tokenized_val_examples)

# Create the model configuration
config = GPT2Config(
    vocab_size=tokenizer.n_vocab,
    n_ctx=max_length,
    n_embd=768,
    n_layer=12,
    n_head=12,
)

# Initialize GPT-2 model
model = GPT2LMHeadModel(config)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=6,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",
)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

model.save_pretrained("pModel")


2024-07-30 09:28:50.636652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 09:28:50.636795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 09:28:50.776378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split: 0 examples [00:00, ? examples/s]

Max length: 39


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,7.470938
2,No log,6.857224
3,No log,6.376606
4,No log,5.931629
5,No log,5.476972
6,No log,4.797491
7,No log,4.596950
8,No log,3.995979
9,No log,3.562976
10,No log,3.143193


### Load Model

In [ ]:
from transformers import GPT2LMHeadModel

# Load the model
model = GPT2LMHeadModel.from_pretrained("pModel")

### Load Tokenizer

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
# from datasets import load_dataset
# from sklearn.model_selection import train_test_split

# # Initialize the tokenizer
# tokenizer = tiktoken.encoding_for_model('gpt-4o')

# # Load dataset
# # file_path = "/kaggle/input/topics22k/_topics.txt"
# file_path = "/kaggle/input/500-mult/500.txt"

# dataset = load_dataset('text', data_files=file_path)
# texts = dataset['train']['text']

# # Split the dataset
# train_texts, val_texts = train_test_split(texts, test_size=0.1, random_state=42)

# # Function to tokenize examples and pad sequences manually
# def tokenize_and_pad(examples, max_length):
#     tokenized_examples = []
#     for example in examples:
#         tokens = tokenizer.encode(example)
#         attention_mask = [1] * len(tokens)
#         padded_tokens = tokens + [0] * (max_length - len(tokens))  # Pad with zeros
#         padded_attention_mask = attention_mask + [0] * (max_length - len(attention_mask))
#         tokenized_examples.append({
#             "input_ids": padded_tokens,
#             "attention_mask": padded_attention_mask,
#             "labels": padded_tokens.copy()  # Use input_ids as labels for language modeling
#         })

#     return tokenized_examples

# # Determine maximum sequence length
# max_length = max(len(tokenizer.encode(example)) for example in texts)
# print("Max length:", max_length)

# # Tokenize and pad train and validation datasets manually
# tokenized_train_examples = tokenize_and_pad(train_texts, max_length)
# tokenized_val_examples = tokenize_and_pad(val_texts, max_length)

# # Create a custom dataset class
# class CustomDataset(Dataset):
#     def __init__(self, tokenized_examples):
#         self.examples = tokenized_examples

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val) for key, val in self.examples[idx].items()}
#         return item

# train_dataset = CustomDataset(tokenized_train_examples)
# val_dataset = CustomDataset(tokenized_val_examples)

# # Create the model configuration
# config = GPT2Config(
#     vocab_size=tokenizer.n_vocab,
#     n_ctx=max_length,
#     n_embd=768,
#     n_layer=12,
#     n_head=12,
# )

# # Initialize GPT-2 model
# model = GPT2LMHeadModel(config)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     overwrite_output_dir=True,
#     num_train_epochs=10,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=6,
#     save_steps=10_000,
#     save_total_limit=2,
#     evaluation_strategy="epoch",
# )

# # Trainer instance
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

# # Train the model
# trainer.train()

# model.save_pretrained("pModel2")

Generating train split: 0 examples [00:00, ? examples/s]

Max length: 168


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.974708
2,No log,0.913560
3,0.959700,0.874582
4,0.959700,0.839392
5,0.959700,0.809894
6,0.692000,0.793854
7,0.692000,0.785285
8,0.692000,0.777782
9,0.551300,0.774415
10,0.551300,0.774401


In [ ]:

from transformers import GPT2LMHeadModel

# Load the model
model = GPT2LMHeadModel.from_pretrained("pModel")

# Next word prediction function
def predict_next_word(model, tokenizer, text, max_length):
    model.eval()
    device = next(model.parameters()).device  # Get the device of the model
    tokens = tokenizer.encode(text)
    input_ids = torch.tensor(tokens).unsqueeze(0).to(device)  # Add batch dimension and move to the same device as the model
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])
    return predicted_token

# Sentence prediction function
def predict_sentence(model, tokenizer, text, max_length, num_words=10):
    model.eval()
    device = next(model.parameters()).device  # Get the device of the model
    tokens = tokenizer.encode(text)
    input_ids = torch.tensor(tokens).unsqueeze(0).to(device)  # Add batch dimension and move to the same device as the model
    generated = tokens
    for _ in range(num_words):
        with torch.no_grad():
            outputs = model(input_ids)
            predictions = outputs.logits
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        generated.append(predicted_index)
        input_ids = torch.tensor(generated).unsqueeze(0).to(device)  # Add batch dimension and move to the same device as the model
    predicted_sentence = tokenizer.decode(generated)
    return predicted_sentence

# Example usage
text = "हाफकिन संस्थेम"
text = "Nara Lokesh said"
text = "बता दें कि कांग्रेस के दिग्गज नेता जवाह"
print("Next word prediction:", predict_next_word(model, tokenizer, text, max_length))
print("Sentence prediction:", predict_sentence(model, tokenizer, text, max_length, num_words=10))

Next word prediction: ल
Sentence prediction: बता दें कि कांग्रेस के दिग्गज नेता जवाहलाल नेहरू के बाद पीएम मोदी दूसरे ऐसे


In [ ]:
# import shutil


# # Compress the entire working directory
# shutil.make_archive('working_directory', 'zip', '/kaggle/working/pModel')


# from IPython.display import FileLink


# # Create a link to download the file
# FileLink('working_directory.zip')
